In [0]:
import os

root_dir = '/content/'
os.chdir(root_dir)

!ls -al

Create Kaggle API token.
Upload the token 'kaggle.json' at '/content/kaggle' directory.

In [0]:
os.environ['KAGGLE_CONFIG_DIR'] = root_dir

In [0]:
!kaggle competitions download -c denoising-dirty-documents

In [0]:
train = os.path.join(root_dir, 'train.zip')
train_cleaned = os.path.join(root_dir, 'train_cleaned.zip')
test = os.path.join(root_dir, 'test.zip')

In [0]:
from zipfile import ZipFile 

def extract_zip_file(file_names):
    for file in file_names:
        with ZipFile(file, 'r') as zip: 
            zip.extractall() 

extract_zip_file([train, train_cleaned, test])

In [0]:
import glob

train_images = sorted(glob.glob('train/*.png'))
train_cleaned_images = sorted(glob.glob('train_cleaned/*.png'))
test_images = sorted(glob.glob('test/*.png'))

In [0]:
import numpy as np
np.random.seed(7)

In [0]:
from keras.preprocessing.image import load_img, array_to_img, img_to_array

def convert_imgs_to_array(images_folder, test=False):
    images = []
    for img_dir in images_folder:
        image = load_img(img_dir, color_mode='grayscale', target_size=(258, 540, 1))
        image = img_to_array(image).astype('float32') / 255.0
        images.append(image)
    return np.asarray(images)

X_train_full = convert_imgs_to_array(train_images)
Y_train_full = convert_imgs_to_array(train_cleaned_images)

In [0]:
import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X_train_full, Y_train_full, test_size=0.2, random_state=42)

In [0]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input

def build_autoenocder():
    input_img = Input(shape=(None,None,1), name='image_input')
    
    #enoder 
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='Conv1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv2')(x)
    x = MaxPooling2D((2,2), padding='same', name='pool1')(x)
    
    #decoder
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv3')(x)
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='Conv4')(x)
    x = UpSampling2D((2,2), name='upsample2')(x)
    x = Conv2D(1, (3,3), activation='sigmoid', padding='same', name='Conv5')(x)
    
    #model
    autoencoder = Model(inputs=input_img, outputs=x)
    return autoencoder

model = build_autoenocder()
model.summary()

In [0]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [0]:
K = keras.callbacks
reduce_lr = K.ReduceLROnPlateau(monitor='val_mae', patience=7, verbose=1, factor=0.1, min_lr=0.00001)
early_stopping = K.EarlyStopping(monitor='val_mae', patience=20, restore_best_weights=True, verbose=1, mode='auto')

In [0]:
epochs = 500
batch_size = 2
history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), 
                   batch_size=batch_size, epochs=epochs, shuffle=True,
                   callbacks=[reduce_lr, early_stopping])

In [0]:
val_loss, val_mae = model.evaluate(X_val, Y_val, verbose=1)
print("Validation loss: {0:.4f} and mae: {1:.4f}".format(val_loss, val_mae))

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline 

import cv2

def plot_documents_after_training(nrows=3, ncols=2):
    selection = np.random.choice(len(test_images), size=(nrows*ncols), replace=False)
    images = np.asarray(test_images)[selection]

    fig, axes = plt.subplots(figsize=(nrows*20, ncols*30), nrows=nrows, ncols=ncols)
    fig.subplots_adjust(hspace=.05, wspace=.05)
    axes = axes.ravel()
    for img, i in zip(images, range(0, nrows*ncols, 2)):
        original_img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
        img = load_img(img, color_mode='grayscale', target_size=(original_img.shape[0], original_img.shape[1], 1))
        img = img_to_array(img).astype('float32') / 255.0
        predicted_img = model.predict(np.expand_dims(img, axis=0))
        axes[i].imshow(array_to_img(img), cmap='gray')
        axes[i+1].imshow(array_to_img(predicted_img[0]), cmap='gray')
        axes[i].axis('off')
        axes[i+1].axis('off')
        
plot_documents_after_training()